<p align="center">
  <img src="logo-icon-color--white@3.png" alt="Groq Logo" width="180" />
</p>

# Groq + Browserbase MCP Demo

Ever wanted to control browsers, extract data, or test sites—just by describing what needs to be done, and at a scale that only the latest AI hardware can handle?

This cookbook walks you through practical, hands-on demos for using Groq's lightning-fast inference with the Browserbase Model Context Protocol (MCP) server. 

### Why Browserbase MCP?
Browserbase MCP turns natural language instructions into real actions across websites.

Whether scraping ecommerce data, monitoring page changes, testing apps, or extracting leads—these recipes demonstrate how Groq and Browserbase MCP let modern AI tackle web tasks at scale.

### What You’ll Learn
- How to use natural language to direct browser automation
- Extracting structured data instantly—from anywhere on the web
- Managing sessions, handling authentication, and building robust browser automation chains
- Real use-case examples you can run or adapt for your own projects


## Quick Setup

Browserbase's MCP is hosted on Smithery.ai (a hosted MCP endpoint). Authenticate there to obtain an MCP server URL for this notebook.

1. Sign up for a free Browserbase account: [browserbase.com](https://www.browserbase.com/)
2. Go to the Browserbase MCP on Smithery: [smithery.ai/server/@browserbasehq/mcp-browserbase](https://smithery.ai/server/@browserbasehq/mcp-browserbase)
3. Connect your Browserbase project:
   - Enter your Browserbase Project ID and API key
   - Authenticate to generate an MCP server URL
4. Copy the MCP URL from Smithery (Click "Get URL with Keys instead")
5. Export it as an environment variable:
   - macOS/Linux: `export SMITHERY_MCP_URL='paste_url_here'`
   - Windows (PowerShell): `$Env:SMITHERY_MCP_URL='paste_url_here'`
6. Ensure your Groq API key is set: `export GROQ_PROD_KEY='your_groq_key'` (or via your secrets manager)

Once set, this notebook will use `SMITHERY_MCP_URL` to connect securely to the Browserbase MCP via Smithery.

In [ ]:
#!/usr/bin/env python3
"""
Groq + Browserbase MCP Demo
"""

import json
import os
import time
from datetime import datetime

from openai import OpenAI

# API Configuration
GROQ_PROD_KEY = os.getenv("GROQ_PROD_KEY")
SMITHERY_MCP_URL = os.getenv("SMITHERY_MCP_URL")

# Check if API keys are set
if not GROQ_PROD_KEY:
    print("Please set your Groq production API key:")
    print("   export GROQ_PROD_KEY='your_key_here'")
    print("   Get it from: https://console.groq.com")
elif not SMITHERY_MCP_URL:
    print("Please set your Smithery hosted MCP URL:")
    print("   export SMITHERY_MCP_URL='your_authenticated_mcp_url_here'")
    print("   Obtain it at: https://smithery.ai/server/@browserbasehq/mcp-browserbase after authenticating with your Browserbase Project ID and API key")
else:
    print("✅ Groq API key configured successfully!")
    print("✅ Smithery hosted MCP URL configured successfully!")
    
# Model configuration
MODEL = "qwen/qwen3-32b" 



## Core Function


In [ ]:
def automate_browser_with_groq(command, url=None):
    """
    Automate browser actions using Groq + Browserbase MCP

    This tutorial function demonstrates how to:
    - Use Groq's ultra-fast LLM inference (500+ tokens/sec)
    - Control a browser via Browserbase MCP (Model Context Protocol) server
    - Execute natural-language-driven automation across the web
    """

    # --- 1. Environment / Safety Checks ---
    # Ensure you have both:
    # - A valid Groq API key
    # - The URL for the Smithery-hosted MCP server (Browserbase backend)
    if not GROQ_PROD_KEY or not SMITHERY_MCP_URL:
        print("❌ Please set both Groq API key and Smithery hosted MCP URL first!")
        return

    # --- 2. Initialize the Groq Client ---
    # The Groq API is OpenAI-compatible, so we use the OpenAI client,
    # but override the base_url to Groq’s API endpoint.
    client = OpenAI(
        base_url="https://api.groq.com/api/openai/v1",
        api_key=GROQ_PROD_KEY
    )

    # --- 3. Display Input for Transparency ---
    print(f"🤖 Command: {command}")
    if url:
        print(f"🌐 Target URL: {url}")

    # Record the start time for performance metrics
    start_time = time.time()

    # --- 4. Configure Browserbase MCP Tool ---
    # We tell Groq that it’s allowed to use the Browserbase MCP tool
    # for executing browser actions. This is passed as part of the `tools` argument.
    tools = [
        {
            "type": "mcp",                  # Use MCP tool type
            "server_url": SMITHERY_MCP_URL, # Smithery MCP endpoint
            "server_label": "browserbase",  # Friendly label (used in logs)
            "require_approval": "never"     # Skip tool-call confirmation
        }
    ]

    # --- 5. Build the Prompt ---
    # If a URL is given, prepend it with a navigation instruction.
    prompt = command
    if url:
        prompt = f"Navigate to {url} and then: {command}"


    # --- 6. Call Groq Responses API ---
    # We now ask Groq to generate output + tool calls.
    response = client.responses.create(
        model=MODEL,       # e.g. "mixtral-8x7b-32768" or another Groq-optimized LLM
        input=prompt,      # Our instruction (with optional URL)
        tools=tools,       # Allow Groq to call Browserbase MCP
        stream=False,      # Disable streaming for simplicity
        temperature=0.1,   # Low randomness → deterministic automation
        top_p=0.4          # Controls token diversity → balanced but focused
    )

    # --- 7. Measure Elapsed Time ---
    total_time = time.time() - start_time

    # --- 8. Extract Response Text ---
    # The response may contain plain text + tool calls.
    content = response.output_text if hasattr(response, "output_text") else str(response)

    # --- 9. Print Results Block ---
    print("=" * 80)
    print("BROWSER AUTOMATION RESULTS")
    print("=" * 80)
    print(content)
    print("=" * 80)

        # --- 10. Inspect Tool Calls (Executed Browser Actions: show name + arguments only) ---
    executed_tools = []

    resp_output = getattr(response, "output", None)
    if resp_output:
        for output_item in resp_output:
            # Each "mcp_call" indicates an action sent to Browserbase MCP
            if getattr(output_item, "type", None) == "mcp_call":
                executed_tools.append(output_item)

    if executed_tools:
        print(f"\n🔧 BROWSERBASE MCP CALLS: Found {len(executed_tools)} browser actions:")
        print("-" * 50)

        for i, tool in enumerate(executed_tools, 1):
            print(f"\nBrowser Action #{i}")
            print("   Type: mcp")
            print(f"   Action: {getattr(tool, 'name', '')}")
            print(f"   Server: {getattr(tool, 'server_label', '')}")

            # Arguments only (no output parsing). Try JSON for readability, but don't fail if it's not JSON.
            args = getattr(tool, "arguments", "{}")
            if isinstance(args, str):
                try:
                    args_to_print = json.loads(args)
                except Exception:
                    args_to_print = args  # keep raw if not JSON
            else:
                args_to_print = args  # already a dict/object

            print(f"   Arguments: {args_to_print}")

    # --- 12. Performance Summary ---
    print("\n📊 PERFORMANCE SUMMARY")
    print(f"   Total time: {total_time:.2f} seconds")
    print(f"   Browser actions: {len(executed_tools)}")

    # --- 13. Return Data for Programmatic Use ---
    return {
        "content": content,
        "total_time": total_time,
        "browser_actions_performed": len(executed_tools),
        "timestamp": datetime.now().isoformat(),
        "url": url,
        "command": command,
    }

## Demo 1: Navigate and Extract Data from a Website

Let's start by navigating to a website and extracting information to see the speed and accuracy in action.


In [ ]:
# Let's extract information from a news website
# This will demonstrate both speed and real-time data extraction

result_news = automate_browser_with_groq(
    "Extract the top 3 headlines and their summaries", 
    "https://news.ycombinator.com"
)


## Next Steps

- **Experiment with variations**  
  Try simple tweaks first: change the target websites, ask for different fields (titles, summaries, ratings), or combine extraction with screenshots for richer context.  

- **Explore multi-step workflows**  
  Chain actions together — e.g., log into a demo account, navigate through a form, extract confirmation details, and store them. This helps you understand how Browserbase MCP maintains sessions across steps.  

- **Build applications**  
  Wrap this function into a script or service. For example:  
  - A scheduled job that checks flight prices every morning  
  - A monitoring service that watches competitor sites for changes  
  - A quick prototype that feeds scraped data into a local dashboard  

- **Scale up**  
  Use Browserbase's multi-session management to launch dozens of browser sessions in parallel. Groq’s LPU-powered inference keeps natural-language instructions fast and responsive, even at scale.  

- **Add structure**  
  Encourage the model to return strict JSON so you can directly feed results into other tools (pandas, databases, dashboards) without manual cleanup.  

- **Visualize results**  
  Use Python libraries like `matplotlib` or `pandas` to turn extracted data into quick comparison charts — weather forecasts, product prices, or sentiment summaries.  

---
## Challenges & Ideas to Try

- **Weather Tracker**  
  Modify the weather example to check *three different cities* instead of one, and return a side-by-side JSON comparison.  

- **News Roundup**  
  Open CNN, BBC, and Reuters. Extract the top headline from each homepage and return them in a structured list.  

- **Shopping Assistant**  
  Instead of full price scraping, just grab the *page titles* from Amazon, Walmart, and Target for a given product search. Compare how different retailers label the same item.  

- **Restaurant Finder**  
  Check Yelp and TripAdvisor for “coffee shops near me.” Extract the top 2 names and ratings.  

- **Screenshot Explorer**  
  Ask Browserbase to capture screenshots of the first 2 search results from Google News and display them inline in your notebook.  

- **Build a Chain**  
  Try a multi-step workflow:  
  1. Go to Wikipedia, search for “Artificial Intelligence.”  
  2. Extract the first paragraph.  
  3. Open YouTube, search for “Artificial Intelligence documentary.”  
  4. Return the first video title.  

These challenges will push you to practice **multi-tab navigation**, **structured extraction**, and **parallel browser sessions** — the building blocks of real-world automation.

---

## Resources

- **Browserbase MCP Server**: [smithery.ai/server/@browserbasehq/mcp-browserbase](https://smithery.ai/server/@browserbasehq/mcp-browserbase)  
- **Browserbase Dashboard**: [browserbase.com/overview](https://www.browserbase.com/overview)  
- **Groq API**: [console.groq.com](https://console.groq.com)  
- **Stagehand Docs**: [docs.stagehand.dev](https://docs.stagehand.dev/)  

---
